In [10]:
import pandas as pd
import numpy as np

dataset_path = 'dataset/50Ktrain.csv'
data = pd.read_csv(dataset_path)
# data = pd.read_csv(dataset_path)
print(data.shape)
# 篩選資料集，若有缺失值則整row刪除

# # 1. 刪除缺失值
# data = data.replace(' ?', np.nan).dropna()
# data = data.replace(' <=50K', 0).replace(' >50K', 1)
#print(data.shape)
#print(data.head(3))

keys_immutable = ['race', 'sex', 'native-country']  # Replace with your selected immutable feature names

# choose data[0] as input feature, for all data points, if the immutable feature of data point isn't same as data[0], then drop it
for key in keys_immutable:
    data = data[data[key] == data.iloc[0][key]]

# drop 'education' and 'fnlwgt' features
data = data.drop(['workclass', 'education', 'fnlwgt', 'occupation', 'race', 'sex', 'marital-status', 'relationship', 'native-country'], axis=1)

print(data.shape)

data = data.reset_index(drop=True)
print(data.head(3))

(32561, 15)
(17653, 6)
   age  education-num  capital-gain  capital-loss  hours-per-week  50K
0   39             13          2174             0              40    0
1   50             13             0             0              13    0
2   38              9             0             0              40    0


In [12]:
# data standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# standardization all columns except the last column
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
# covert to dataframe
df = pd.DataFrame(data)
print(df.head(3))



# K-means clustering
from sklearn.cluster import KMeans
x = df.iloc[:, :-1] # 最後一項不要 (income) 其他去做kmeans
model = KMeans(n_clusters=100, n_init='auto', random_state=1).fit(x)

# get centers of each cluster
centers = model.cluster_centers_
print(centers)



        age  education-num  capital-gain  capital-loss  hours-per-week  50K
0 -0.058610       1.104109      0.088997     -0.238769       -0.222405    0
1  0.758526       1.104109     -0.163767     -0.238769       -2.418858    0
2 -0.132895      -0.510674     -0.163767     -0.238769       -0.222405    0
[[-1.53599194e+00 -8.30401267e-01 -1.56015973e-01 -2.38769059e-01
  -1.21617798e+00]
 [-3.87552957e-01  1.36172904e-03 -1.13047087e-01 -2.38769059e-01
   5.10005623e-01]
 [-5.84883284e-01 -3.96957048e-01 -1.63766762e-01  4.57756860e+00
  -1.80011457e-01]
 [ 9.81381266e-01  1.61545712e+00  1.14627655e+01 -2.38769059e-01
   1.37748011e+00]
 [ 1.54118984e+00 -4.20963998e-01 -1.18740476e-01 -2.38769059e-01
  -1.69308735e+00]
 [ 7.22277466e-01 -1.77589368e-02 -1.43222449e-01 -2.38769059e-01
  -2.06028345e-01]
 [ 6.33783099e-01 -3.31676978e-01 -1.18055387e-01 -2.38769059e-01
   2.62868555e+00]
 [ 2.50471499e-01  1.13202824e+00 -1.54454577e-01 -2.38769059e-01
  -2.59506897e-01]
 [ 2.41906939e-0

In [ ]:
# 透過Elbow法找出最佳K值(資料群數)
Dist = []
K = range(1,18)
for k in K:
    kmeanModel = KMeans(n_clusters=k, n_init='auto', random_state=1).fit(x)
    kmeanModel.fit(x)
    Dist.append(kmeanModel.inertia_)
print(Dist)
plt.plot(range(1,18), Dist, 'bx-')
plt.show()



In [125]:
import pandas as pd
import queue
from sklearn.neighbors import NearestNeighbors
import numpy as np

q = queue.Queue()
q.put(1)

while not q.empty():
    index = q.get()

    df = pd.DataFrame(data)

    # Extract features for KNN
    features = df[[0, 1, 4]]

    # choose data[0] as target data point for which you want to find neighbors
    target_data_point = features.iloc[index]

    # Create a KNN model
    knn_model = NearestNeighbors(n_neighbors=6, algorithm='auto')  # Adjust n_neighbors as needed

    # Fit the model with your dataset
    knn_model.fit(features)

    # Find the indices of the nearest neighbors (excluding the target_data_point itself)
    _, indices = knn_model.kneighbors([target_data_point])
    neighbor_indices: np.ndarray = indices[0]
    neighbor_indices = np.delete(neighbor_indices, np.where(neighbor_indices == index))

    # for i in neighbor_indices:
    #     q.put(i)

    # Print the neighbors
    print("Nearest neighbors:")
    print(df.iloc[neighbor_indices])

    # store the index of the nearest neighbors
    neighbor_indices = neighbor_indices.tolist()

Nearest neighbors:
              0         1         2         3         4        5
15081  0.810470  1.097452 -0.165421 -0.241688 -2.961453 -0.70739
10727  1.197121  1.097452 -0.165421 -0.241688 -2.372177 -0.70739
8454   0.423820  1.097452 -0.165421 -0.241688 -2.372177 -0.70739
4792   0.887800  1.097452 -0.165421 -0.241688 -2.961453 -0.70739
2774   0.346489  1.097452 -0.165421 -0.241688 -2.624724 -0.70739


In [ ]:
# generate a cost function that takes in two data point and returns a cost table 
# the cost table is the cost of the two data point to each other
def generate_cost_table(data_point_1, data_point_2):
# use their features to calculate the cost by a polynomial function
    cost = 0
    for i in range(len(data_point_1)):
        cost += (data_point_1[i] - data_point_2[i]) ** 2
    return cost